# Import libraries

In [1]:
import os
from pathlib import Path
import shutil
from pydub import AudioSegment

In [2]:
# Define paths
DEST_DIR = "../data/converted/TSync2-to-vctk"
DEST_TEXT_PATH = os.path.join(DEST_DIR, "txt/TSync2")
DEST_AUDIO_PATH = os.path.join(DEST_DIR, "wav32/TSync2")
SRC_AUDIO_PATH = "../data/raw/TSync2/wav"
SRC_TEXT_PATH = "../data/raw/TSync2/wrd_ph"

# Clean and create directories
if os.path.exists(DEST_DIR):
   print("Clearing destination folder")
   shutil.rmtree(DEST_DIR)
os.makedirs(DEST_TEXT_PATH, exist_ok=True)
os.makedirs(DEST_AUDIO_PATH, exist_ok=True)

wavCount = 0
skipWav = []
# Convert audio files from Tsync to VCTK format
for file in sorted(Path(SRC_AUDIO_PATH).glob("*.wav")):  # Only process .wav files
   #  data, samplerate = sf.read(file)
   #  print(f"{file.name}: {data.shape}, Sample Rate: {samplerate}")
   src_file = os.path.join(SRC_AUDIO_PATH, file.name)
   try:
      src_wav = AudioSegment.from_wav(src_file)
      wavCount += 1
   except Exception as e:
      print(f"Error processing {file.name}: {e}")
      skipWav.append(wavCount+len(skipWav))
      continue
   if wavCount <= 9:
      dest_file = os.path.join(DEST_AUDIO_PATH, 'Tsync2'+'_00'+str(wavCount)+'_mic1.flac')
   elif wavCount <= 99:
      dest_file = os.path.join(DEST_AUDIO_PATH, 'Tsync2'+'_0'+str(wavCount)+'_mic1.flac')
   else:
      dest_file = os.path.join(DEST_AUDIO_PATH, 'Tsync2'+'_'+str(wavCount)+'_mic1.flac')
   src_wav.export(dest_file, format="flac")
   shutil.copy2(src_file, dest_file)

txtCount = 0
skipNum = 0
#Convert txt files from Tsync to VCTK format
for file in sorted(Path(SRC_TEXT_PATH).glob("*.txt")):  
    with file.open('r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        clean_text = "".join(first_line.split("|"))
        clean_name = file.name.split("_")
        txtCount += 1
        if txtCount in skipWav:
            skipNum += 1
        else:
         if txtCount <= 9:
               file_path = os.path.join(DEST_TEXT_PATH, 'Tsync2'+'_00'+str(txtCount-skipNum)+'.txt')  # Define the file name
         elif txtCount <= 99:
               file_path = os.path.join(DEST_TEXT_PATH, 'Tsync2'+'_0'+str(txtCount-skipNum)+'.txt')
         else:
               file_path = os.path.join(DEST_TEXT_PATH, 'Tsync2'+'_'+str(txtCount-skipNum)+'.txt')
         with open(file_path, 'w', encoding='utf-8') as file:
               file.write(clean_text)

Clearing destination folder
Error processing tsync2_noon_0_1943.wav: Decoding failed. ffmpeg returned error code: 8

Output from ffmpeg/avlib:

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --ena